# Моделирование LTV 



In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import joblib

In [2]:
PROCESSED_PATH = 'data/processed'
FEATURES_FILE = os.path.join(PROCESSED_PATH, 'features.csv')

df = pd.read_csv(FEATURES_FILE)
print(f"Загружено строк: {len(df):,}")

Загружено строк: 96,091


## Подготовка данных

In [3]:
# Признаки (исключаем строковые и ID)
features = [
    col for col in df.columns if col not in
    ['customer_unique_id', 'future_ltv', 'recency', 'churn', 'rfm_segment']
]
X = df[features].astype(float)
y = df['future_ltv']
y_binary = (y > 0).astype(int)

# Средний чек повторных покупок
avg_return_amount = y[y > 0].mean()
print(f"Средний повторный чек: {avg_return_amount:.2f} $")

print(f"Положительных примеров: {y_binary.sum()} ({y_binary.mean():.4%})")

Средний повторный чек: 140.45 $
Положительных примеров: 10 (0.0104%)


## Классификация возврата (кросс-валидация)

In [4]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
roc_aucs = []
all_prob = np.zeros(len(X))

scale_pos_weight = (y_binary == 0).sum() / max(1, (y_binary == 1).sum()) * 5

# Переменные для сохранения моделей из последнего фолда
last_xgb = None
last_lgbm = None
last_cat = None

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y_binary), 1):

    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr_bin, y_val_bin = y_binary.iloc[train_idx], y_binary.iloc[val_idx]

    xgb = XGBClassifier(scale_pos_weight=scale_pos_weight, random_state=42)
    lgbm = LGBMClassifier(is_unbalance=True, random_state=42, verbose=-1)
    cat = CatBoostClassifier(auto_class_weights='Balanced',
                             random_state=42,
                             verbose=0)

    xgb.fit(X_tr, y_tr_bin)
    lgbm.fit(X_tr, y_tr_bin)
    cat.fit(X_tr, y_tr_bin)

    prob_fold = (xgb.predict_proba(X_val)[:, 1] +
                 lgbm.predict_proba(X_val)[:, 1] +
                 cat.predict_proba(X_val)[:, 1]) / 3

    roc = roc_auc_score(y_val_bin, prob_fold)
    roc_aucs.append(roc)
    all_prob[val_idx] = prob_fold

    # Сохраняем модели из последнего фолда
    if fold == 5:
        last_xgb = xgb
        last_lgbm = lgbm
        last_cat = cat

print(
    f"\nСредний ROC-AUC по 5 фолдам: {np.mean(roc_aucs):.4f} ± {np.std(roc_aucs):.4f}"
)


Средний ROC-AUC по 5 фолдам: 0.8383 ± 0.1943


In [5]:
# Сохранение моделей из последнего фолда
models = {'xgb': last_xgb, 'lgbm': last_lgbm, 'cat': last_cat}

joblib.dump(models, 'ltv_clf_models_last_fold.pkl')
print("Модели из последнего фолда сохранены: ltv_clf_models_last_fold.pkl")

Модели из последнего фолда сохранены: ltv_clf_models_last_fold.pkl


## Итоговый LTV

In [6]:
y_pred_ltv = all_prob * avg_return_amount

print("Статистика предсказанного LTV (по кросс-валидации):")
print(pd.Series(y_pred_ltv).describe())

print(f"\nСредний предсказанный LTV: {y_pred_ltv.mean():.2f} $")
print(f"Доля >0: {(y_pred_ltv > 0).mean():.2%}")

Статистика предсказанного LTV (по кросс-валидации):
count    96091.000000
mean        20.490334
std         23.267426
min          0.000002
25%          0.000080
50%          0.000952
75%         46.817032
max        140.381236
dtype: float64

Средний предсказанный LTV: 20.49 $
Доля >0: 100.00%


## Сегментация в 10 групп 

In [7]:
df['pred_ltv'] = y_pred_ltv
df['decile'] = pd.qcut(df['pred_ltv'],
                       q=10,
                       labels=range(1, 11),
                       duplicates='drop')
df['segment'] = pd.cut(df['pred_ltv'],
                       bins=10,
                       labels=[
                           '1 (самые низкие)', '2', '3', '4', '5', '6', '7',
                           '8', '9', '10 (топ 10%)'
                       ])

segment_stats = df.groupby('segment')['pred_ltv'].agg(
    ['count', 'mean', 'min', 'max']).round(2)
segment_stats['доля_клиентов'] = segment_stats['count'] / len(df)
print("\nСтатистика по 10 сегментам:")
print(segment_stats)


Статистика по 10 сегментам:
                  count    mean     min     max  доля_клиентов
segment                                                       
1 (самые низкие)  54130    0.02    0.00   13.25       0.563320
2                    11   20.21   14.07   27.20       0.000114
3                     3   35.78   33.49   40.36       0.000031
4                 41877   46.85   46.82   55.44       0.435806
5                    35   62.88   56.64   69.70       0.000364
6                    18   77.68   71.46   84.01       0.000187
7                    13   90.38   84.63   96.27       0.000135
8                     2  106.93  103.13  110.74       0.000021
9                     0     NaN     NaN     NaN       0.000000
10 (топ 10%)          2  134.10  127.82  140.38       0.000021


/var/folders/h2/dsmg5l1j0ts5vnwjkk8_hcy40000gn/T/ipykernel_23425/1138731032.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  segment_stats = df.groupby('segment')['pred_ltv'].agg(


# Рекомендации по акциям

In [8]:
recommendations = {
    '1 (самые низкие)':
    "Минимальные усилия: общие рассылки 1 раз в квартал",
    '2':
    "Лёгкие напоминания: скидка 5% на следующую покупку",
    '3':
    "Базовые акции: скидка 10%, email с рекомендациями",
    '4':
    "Персонализация: рекомендации по категориям + скидка 15%",
    '5':
    "Средний уровень: кросс-продажи, бандлы, скидка 20%",
    '6':
    "Активное удержание: персональные предложения, cashback",
    '7':
    "Высокий приоритет: VIP-скидки 25–30%, ранний доступ к новинкам",
    '8':
    "Премиум: персональный менеджер, эксклюзивные акции",
    '9':
    "Топ-клиенты: бонусы за лояльность, подарки, закрытые распродажи",
    '10 (топ 10%)':
    "Максимум внимания: индивидуальные предложения, A/B-тесты, VIP-события"
}

print("\nРекомендации по акциям:")
for segment, rec in recommendations.items():
    print(f"{segment}: {rec}")


Рекомендации по акциям:
1 (самые низкие): Минимальные усилия: общие рассылки 1 раз в квартал
2: Лёгкие напоминания: скидка 5% на следующую покупку
3: Базовые акции: скидка 10%, email с рекомендациями
4: Персонализация: рекомендации по категориям + скидка 15%
5: Средний уровень: кросс-продажи, бандлы, скидка 20%
6: Активное удержание: персональные предложения, cashback
7: Высокий приоритет: VIP-скидки 25–30%, ранний доступ к новинкам
8: Премиум: персональный менеджер, эксклюзивные акции
9: Топ-клиенты: бонусы за лояльность, подарки, закрытые распродажи
10 (топ 10%): Максимум внимания: индивидуальные предложения, A/B-тесты, VIP-события
